# Finetuning PubmedBERT for citation intent classification

## Data preprocessing

In [ ]:
from datasets import Dataset
import pandas as pd

df = pd.read_csv('../data/software_citation_intent_merged.csv')
df

,Unnamed: 0,id,sentence,used,created,mention,context,label,text
0,0,PMC5189946,All of this analysis was implemented using Mat...,False,True,False,NaN,0,All of this analysis was implemented using Mat...
1,1,PMC4511233,"Code for calculating partition similarity, obt...",False,True,False,Since the probability of getting a given MI is...,0,"Code for calculating partition similarity, obt..."
2,2,PMC4186879,All behavioral statistical analyses were perfo...,False,False,True,All behavioral statistical analyses were perfo...,2,All behavioral statistical analyses were perfo...
3,3,PMC5026371,"M-Track was written using Python 2.7, OpenCV 3...",True,False,False,NaN,1,"M-Track was written using Python 2.7, OpenCV 3..."
4,4,PMC1283974,"Mindboggle is a freely downloadable, open sour...",False,True,False,"Mindboggle is a freely downloadable, open sour...",0,"Mindboggle is a freely downloadable, open sour..."
...,...,...,...,...,...,...,...,...,...
4183,995,PMC2936424,Study sample\n,False,False,False,NaN,3,Study sample\n
4184,996,PMC3660501,U0216 and Wortmannin were dissolved in PBS whe...,False,False,False,NaN,3,U0216 and Wortmannin were dissolved in PBS whe...
4185,997,PMC4213368,The linker unit serves as an acceptor for the ...,False,False,False,NaN,3,The linker unit serves as an acceptor for the ...
4186,998,PMC4451992,The Training and Recourse Center of CIDI in Be...,False,False,False,NaN,3,The Training and Recourse Center of CIDI in Be...


In [ ]:
dataset = df[['text', 'label']]
dataset = dataset.sample(n=len(dataset), random_state=42)
dataset = dataset.reset_index(drop=True)
dataset

,text,label
0,All statistical analyses were performed with S...,1
1,"The subjects underwent prenatal follow-up, and...",3
2,The data were analyzed with SPSS software (IBM...,1
3,We developed an automated image analysis algor...,0
4,Categorizing the free response answers involve...,3
...,...,...
4183,These studies have not been included in the pr...,3
4184,Hierarchical Bayesian parameter estimation usi...,1
4185,°C and tools offered by all providers through ...,2
4186,It measures land use intensity only of the ind...,3


In [ ]:
id2label = {0: "created", 1: "used", 2: "mention", 3: "none"}
label2id = {"created": 0, "used": 1, "mention": 2, "none": 3}

In [ ]:
dataset = Dataset.from_pandas(dataset)
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 4188
})

In [ ]:
type(dataset)

datasets.arrow_dataset.Dataset

In [ ]:
dataset = dataset.train_test_split(test_size=0.2)

In [ ]:
type(dataset)

datasets.dataset_dict.DatasetDict

In [ ]:
len(dataset['test'])

838

In [ ]:
dataset['train'][0]

{'text': 'Thus, an additional MRI model has been included in the DCE@urLAB application and is discussed later.\n',
 'label': 0}

In [ ]:
from transformers import *

tokenizer = AutoTokenizer.from_pretrained('microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext')


[2023-10-27 16:24:06,491] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/databricks/python/lib/python3.10/site-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(
/databricks/python/lib/python3.10/site-packages/transformers/generation_tf_utils.py:24: FutureWarning: Importing `TFGenerationMixin` from `src/transformers/generation_tf_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import TFGenerationMixin` instead.
  warnings.warn(
Loading custom CUDA kernels...
Loading custom CUDA kernels...
Using /root/.cache/torch_extensions/py310_cu118 as PyTorch extensions root...
Creating extension directory /root/.cache/torch_extensions/py310_cu118/cuda_kernel...
Detected CUDA files, patching ldflags
Emitting ninja build file /root/.cache/torch_extensions/py310_cu118/cuda_kernel/build.ninja...
Building extension mo

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext/snapshots/eaa409b6b7c9380a5f2ba7a59aa97712ff30f386/config.json
Model config BertConfig {
  "_name_or_path": "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}



loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--microsoft--BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext/snapshots/eaa409b6b7c9380a5f2ba7a59aa97712ff30f386/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--microsoft--BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext/snapshots/eaa409b6b7c9380a5f2ba7a59aa97712ff30f386/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext/snapshots/eaa409b6b7c9380a5f2ba7a59aa97712ff30f386/config.json
Model config BertConfig {
  "_name_or_path": "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_

### preprocessing - truncation

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
tokenizer_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/3350 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/838 [00:00<?, ? examples/s]

### preprocessing - padding

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

## Model training

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
# model = BertForSequenceClassification.from_pretrained('allenai/scibert_scivocab_uncased', num_labels=4,)
model = AutoModelForSequenceClassification.from_pretrained(
    'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext', num_labels=4, id2label=id2label, label2id=label2id
)

# model = AutoModelForSequenceClassification.from_pretrained(
#     "distilbert-base-uncased", num_labels=4, id2label=id2label, label2id=label2id
# )

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--microsoft--BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext/snapshots/eaa409b6b7c9380a5f2ba7a59aa97712ff30f386/config.json
Model config BertConfig {
  "_name_or_path": "microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "created",
    "1": "used",
    "2": "mention",
    "3": "none"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "created": 0,
    "mention": 2,
    "none": 3,
    "used": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.31.0",
  "t

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--microsoft--BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext/snapshots/eaa409b6b7c9380a5f2ba7a59aa97712ff30f386/pytorch_model.bin
Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initiali

In [ ]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
model.device

device(type='cpu')

In [ ]:
device = 'cuda:0'
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
model.device

device(type='cuda', index=0)

In [ ]:
training_args = TrainingArguments(
    output_dir="./tmp/",
    learning_rate=2e-5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    num_train_epochs=20,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    # load_best_model_at_end=True,
    save_strategy="no",
    # push_to_hub=True,
    
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenizer_dataset["train"],
    eval_dataset=tokenizer_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Found safetensors installation, but --save_safetensors=False. Safetensors should be a preferred weights saving format due to security and performance reasons. If your model cannot be saved by safetensors please feel free to open an issue at https://github.com/huggingface/safetensors!
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/databricks/python/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in 

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.363615,0.884248
2,0.378700,0.351738,0.903341
3,0.182300,0.446255,0.912888
4,0.182300,0.512782,0.906921
5,0.087400,0.519526,0.911695
6,0.046300,0.600673,0.909308
7,0.046300,0.633182,0.910501
8,0.024900,0.622202,0.916468
9,0.008500,0.632987,0.910501
10,0.008500,0.718445,0.905728


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 838
  Batch size = 10
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 838
  Batch size = 10
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 838
  Batch size = 10
The following co

TrainOutput(global_step=6700, training_loss=0.055254011510718445, metrics={'train_runtime': 751.9343, 'train_samples_per_second': 89.104, 'train_steps_per_second': 8.91, 'total_flos': 2464994012692480.0, 'train_loss': 0.055254011510718445, 'epoch': 20.0})

## Validation on CZI dataset

In [ ]:
from datasets import Dataset
import pandas as pd

df_evaluate = pd.read_csv('../data/czi_val_merged.csv')
df_evaluate

,doi_link,source,text,software,intent,label
0,https://doi.org/10.1371/journal.pone.0043845,10. Confocal microscopy,Confocal laser scanning was performed on a Oly...,Fluoview,used,usage
1,https://doi.org/10.1155/2014/395212,5. Simulation Results and Analysis,"In Figure 11, p miss of SVM-SMP is nearly equa...",SVM,used,usage
2,https://doi.org/10.3389/fphys.2020.587057,Network Architecture and Training Parameters,A pattern recognition network was created usin...,MATLAB,used,usage
3,https://doi.org/10.1186/s13046-018-0997-7,paper_abstract,"Finally, we investigated the potential molecul...",analysis (GSEA),used,usage
4,https://doi.org/10.7717/peerj.9470,Conclusions,The GitHub repository for this study has been ...,Python,used,usage
...,...,...,...,...,...,...
405,https://joss.theoj.org/papers/10.21105/joss.05556,title,PASCal Python: A Principal Axis Strain Calculator,PASCal Python,creation,creation
406,https://joss.theoj.org/papers/10.21105/joss.05313,title,QuaC: A Pipeline Implementing Quality Control ...,QuaC,creation,creation
407,https://joss.theoj.org/papers/10.21105/joss.05251,title,ReSurfEMG: A Python library for preprocessing ...,ReSurfEMG,creation,creation
408,https://joss.theoj.org/papers/10.21105/joss.05562,title,DARTS: The Data Analysis Remote Treatment Service,DARTS,creation,creation


In [ ]:
text = df_evaluate['text'].values.tolist()

In [ ]:
from transformers import pipeline

model = model.to('cpu')
classifier = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)
classifier(text)

Disabling tokenizer parallelism, we're using DataLoader multithreading already


[{'label': 'used', 'score': 0.9999843835830688},
 {'label': 'used', 'score': 0.6080527901649475},
 {'label': 'mention', 'score': 0.9999368190765381},
 {'label': 'used', 'score': 0.9999810457229614},
 {'label': 'created', 'score': 0.9999082088470459},
 {'label': 'used', 'score': 0.9999852180480957},
 {'label': 'used', 'score': 0.9999854564666748},
 {'label': 'used', 'score': 0.9336268901824951},
 {'label': 'used', 'score': 0.9999855756759644},
 {'label': 'used', 'score': 0.9999831914901733},
 {'label': 'used', 'score': 0.999984622001648},
 {'label': 'used', 'score': 0.9999843835830688},
 {'label': 'used', 'score': 0.9999841451644897},
 {'label': 'used', 'score': 0.9999858140945435},
 {'label': 'used', 'score': 0.9999858140945435},
 {'label': 'used', 'score': 0.999925971031189},
 {'label': 'used', 'score': 0.999985933303833},
 {'label': 'used', 'score': 0.9999855756759644},
 {'label': 'used', 'score': 0.9999858140945435},
 {'label': 'used', 'score': 0.9999858140945435},
 {'label': 'used'

In [ ]:
map_label = {'usage':'used', 'creation':'created', 'none': 'none', 'mention': 'mention'}
df_evaluate = df_evaluate[['text', 'label']]
df_evaluate['label'] = df_evaluate['label'].apply(lambda x: map_label[x])

/root/.ipykernel/2327/command-2884248819784294-2481495275:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_evaluate['label'] = df_evaluate['label'].apply(lambda x: map_label[x])


In [ ]:
df_evaluate

,text,label
0,Confocal laser scanning was performed on a Oly...,used
1,"In Figure 11, p miss of SVM-SMP is nearly equa...",used
2,A pattern recognition network was created usin...,used
3,"Finally, we investigated the potential molecul...",used
4,The GitHub repository for this study has been ...,used
...,...,...
405,PASCal Python: A Principal Axis Strain Calculator,created
406,QuaC: A Pipeline Implementing Quality Control ...,created
407,ReSurfEMG: A Python library for preprocessing ...,created
408,DARTS: The Data Analysis Remote Treatment Service,created


In [ ]:
import tqdm
cnt = 0
classifier_list = classifier(text)
for i in tqdm.tqdm(range(len(classifier_list))):
    if classifier_list[i]['label'] == df_evaluate.loc[i, 'label']:
        cnt += 1
    else:
        print('-'*80)
        print('[text]', df_evaluate.loc[i, 'text'], '[label]:', df_evaluate.loc[i, 'label'])
        print('[model label]:', classifier_list[i]['label'], '[pred score]:', classifier_list[i]['score'])
print('-'*80)
print('correct:', cnt, 'total:', len(df_evaluate))
print('accuracy:', cnt/len(df_evaluate))


100%|██████████| 410/410 [00:00<00:00, 47070.25it/s]

--------------------------------------------------------------------------------
[text] A pattern recognition network was created using MATLAB r2017b to study the performance of the extracted gait parameters [label]: used
[model label]: mention [pred score]: 0.9999368190765381
--------------------------------------------------------------------------------
[text] The GitHub repository for this study has been created, including the Python scripts and the datasets used in this study (see https://github.com/smiile8888/rna-splice-sites-recognition) [label]: used
[model label]: created [pred score]: 0.9999082088470459
--------------------------------------------------------------------------------
[text] ROI region of interest, STD standard deviation, w/o ROI detection landmark prediction without ROI detection, Mask R-CNN end-to-end model for ROI detection and landmark prediction with Mask R-CNN, w/ ROI detection (segmentation/fixed size or segmentation/variable size or regression /variable

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, multilabel_confusion_matrix, accuracy_score

In [ ]:
classifier_list = classifier(text)
true_labels = [label2id[i] for i in df_evaluate['label'].values.tolist()]
predicted_labels = [label2id[i['label']] for i in classifier_list]

precision = precision_score(true_labels, predicted_labels, average='macro')
recall = recall_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='macro')
accuracy = accuracy_score(true_labels, predicted_labels)
print('precision:', precision)
print('recall:', recall)
print('f1:', f1)
print('accuracy:', accuracy)

precision: 0.27881944444444445
recall: 0.30447941888619856
f1: 0.2885055687065854
accuracy: 0.824390243902439


### Validation on CZI dataset - each label

In [ ]:
confusion_matrices = multilabel_confusion_matrix(true_labels, predicted_labels)
confusion_matrices


array([[[398,   2],
        [ 10,   0]],

       [[ 34,  22],
        [ 24, 330]],

       [[345,  37],
        [ 20,   8]],

       [[381,  11],
        [ 18,   0]]])

Confusion matrix:
[[TN, FP]
 [FN, TP]]

In [ ]:
# Print the results
for label in range(len(label2id)):
    print('-'*80)
    print(f"Label: {id2label[label]}")
    print("Precision:", precision[label])
    print("Recall:", recall[label])
    print("F1 Score:", f1[label])
    print("Confusion Matrix:")
    print(confusion_matrices[label])

--------------------------------------------------------------------------------
Label: created
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
Confusion Matrix:
[[398   2]
 [ 10   0]]
--------------------------------------------------------------------------------
Label: used
Precision: 0.9375
Recall: 0.9322033898305084
F1 Score: 0.9348441926345609
Confusion Matrix:
[[ 34  22]
 [ 24 330]]
--------------------------------------------------------------------------------
Label: mention
Precision: 0.17777777777777778
Recall: 0.2857142857142857
F1 Score: 0.2191780821917808
Confusion Matrix:
[[345  37]
 [ 20   8]]
--------------------------------------------------------------------------------
Label: none
Precision: 0.0
Recall: 0.0
F1 Score: 0.0
Confusion Matrix:
[[381  11]
 [ 18   0]]


## Validation on test dataset

In [ ]:
test = dataset['test']
test_text = [i['text'] for i in test]
test_text

['However, data from Haiti indicate that only 1/3 of those who receive PEP are fully compliant [19]. ',
 'illuminaio provides the first open-source package for this task, and will promote wider uptake of the IDAT format as a standard for sharing Illumina BeadArray data in public databases, in the same way that the CEL file serves as the standard for the Affymetrix platform.\n',
 'The integration of CBFA within the OptFlux framework for ME enables the utilization of different model formats and standards and the integration with complementary methods for phenotype simulation and visualization of results.\n',
 'CoXpress is released as a package for R.\n',
 'Negative observed-minus-expected values signify that a color form was attacked less than expected and vice versa.',
 'The male-to-female population ratios in age were almost constant from 2000 to 2009.',
 'CoXpress is written in the native R language and has been fully tested on both windows and linux.\n',
 'The genomic locations of ME

In [ ]:
classifier_list = classifier(test_text)

In [ ]:
true_labels = [i['label'] for i in test]
predicted_labels = [label2id[i['label']] for i in classifier_list]

precision = precision_score(true_labels, predicted_labels, average='macro')
recall = recall_score(true_labels, predicted_labels, average='macro')
f1 = f1_score(true_labels, predicted_labels, average='macro')
accuracy = accuracy_score(true_labels, predicted_labels)
print('precision:', precision)
print('recall:', recall)
print('f1:', f1)
print('accuracy:', accuracy)

precision: 0.8722084774881793
recall: 0.8796258843337188
f1: 0.8754759732466008
accuracy: 0.9212410501193318
